In [3]:
!pip install gym

You should consider upgrading via the 'C:\Users\tejas\Github projects\RL - Mental health project\pytorch_env\Scripts\python.exe -m pip install --upgrade pip' command.



  Using cached gym-0.26.2-py3-none-any.whl


In [4]:
!pip install stable-baselines3[extra]

  Using cached torch-2.6.0-cp310-cp310-win_amd64.whl (204.2 MB)
  Using cached pillow-11.1.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl (2.4 kB)
  Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
  Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl (15 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)


You should consider upgrading via the 'C:\Users\tejas\Github projects\RL - Mental health project\pytorch_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install --upgrade shimmy gymnasium

You should consider upgrading via the 'c:\Users\tejas\Github projects\RL - Mental health project\pytorch_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import gymnasium as gym
import gym
import numpy as np
import random
from gym import spaces
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("Mental_Health_Score_Dataframe.csv")

In [8]:
len(df)

128

In [9]:
df.head()

,Emotional_Score,Social_Media_Time_Score,Platform_Score,Usage_Pattern_Score,Impact_Score,Additional_Feature_Score,TFIDF_Score,Mental_Health_Score
0,84,13,20,18,27,38,0,200
1,56,13,10,2,0,35,0,116
2,97,23,19,18,61,34,0,252
3,114,13,19,11,54,63,0,274
4,87,13,19,18,25,10,0,172


In [ ]:
class MentalHealthEnv(gym.Env):
    def __init__(self, df):
        super(MentalHealthEnv, self).__init__()

        #loading the dataset
        self.df = df.copy()
        self.current_index = 0


        #define state: 7 mental health scores (ranging between 0 to 300)
        self.observation_space = spaces.Box(low = 0, high = 300, shape = (7,), dtype = np.float32)

        # defining action space
        self.actions = [
            "Engage in Outdoor activities",
            "Reduce the social media usage",
            "Watch educational videos",
            "Increase mindfulness practicess like yoga",
            "Interact with positive communities",
            "Avoid stress-inducing content",
            "Follow a structured daily routine"
        ]

        self.action_space = spaces.Discrete(len(self.actions))

        #initiating state: random values between 50 and 250 (to avoid zero-starting issues)
        self.state = np.random.uniform(low = 50, high = 250, size = (7,))

        # define action effects (each action affects specific scores)
        self.action_effects = {
            "Engage in Outdoor activities": np.array([10, 5, 0, 8, -5, 7, 0]),
            "Reduce the social media usage": np.array([5, -15, 0, 0, 10, 0, 0]),
            "Watch educational videos": np.array([5, 0, 15, 7, -3, 10, 0]),
            "Increase mindfulness practicess like yoga": np.array([12, 0, 0, 5, -8, 15, 0]),
            "Interact with positive communities": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Avoid stress-inducing content": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Follow a structured daily routine": np.array([10, 0, 0, 0, 7, 8, 0])
        }
        self.state = self.get_state_from_df()


    def get_state_from_df(self):
        # extract the initial state from the dataframe based on the current index
        row = self.df.iloc[self.current_index]
        return np.array(row[:-1])
    
    def step(self, action):
        #apply an action and update mental health scores
        self.state = np.clip(self.state + self.action_effects[self.actions[action]], 0, 300)


        # reward: Improvement in mental health score 
        self.df.at[self.current_index, "Mental_Health_Score"] = np.sum(self.state)
        reward = self.df.at[self.current_index, "Mental_Health_Score"]

        #print(f"State Shape: {self.state.shape}")

        done = np.all(self.state >= 280) #stops if all the score are near 300
        
        return self.state, reward, done, {}

    def reset(self):
        """Move to the next user in the dataset or restart if finished."""
        self.current_index += 1  # Move to the next row
        if self.current_index >= len(self.df):
            self.current_index = 0  # Reset if all users are processed
        self.state = self.get_state_from_df()
        return self.state


In [18]:
env = MentalHealthEnv(df)

### Model Training

In [12]:
from stable_baselines3 import DQN, A2C

In [19]:
dqn_model = DQN("MlpPolicy", env, verbose=1)
dqn_model.learn(total_timesteps=5000)
dqn_model.save("mental_health_dqn_model")

# Train A2C
a2c_model = A2C("MlpPolicy", env, verbose=1)
a2c_model.learn(total_timesteps=5000)
a2c_model.save("mental_health_a2c_model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\tejas\Github projects\RL - Mental health project\pytorch_env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| time/                 |          |
|    fps                | 315      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.73    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 94.9     |
|    value_loss         | 5.35e+03 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 343      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.56    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates    

In [21]:
def evaluate_model(model, env, num_episodes=len(df), max_steps=1):
    total_rewards = []
    action_counts = np.zeros(len(env.actions))
    user_suggestions = []

    for user_id in range(num_episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step_count = 0  # Track steps per episode
        actions_taken = []

        while not done and step_count < max_steps: 
            action, _ = model.predict(state)
            action_counts[action] += 1
            actions_taken.append(env.actions[action])
            state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            step_count += 1

        total_rewards.append(episode_reward)
        user_suggestions.append((user_id + 1, actions_taken))

    return np.mean(total_rewards), np.std(total_rewards), action_counts, user_suggestions


# Load trained models
dqn_model = DQN.load("mental_health_dqn_model")
a2c_model = A2C.load("mental_health_a2c_model")

# Evaluate both models
dqn_mean, dqn_std, dqn_actions, dqn_suggestions = evaluate_model(dqn_model, env)
a2c_mean, a2c_std, a2c_actions, a2c_suggestions = evaluate_model(a2c_model, env)

In [22]:
dqn_mean

np.float64(24.59375)

In [23]:
dqn_std

np.float64(3.6302494318572656)

In [24]:
a2c_mean

np.float64(33.2734375)

In [25]:
a2c_std

np.float64(3.109247406301684)

In [30]:
# print("\n🔵 DQN Model Suggestions:")
# for user_id, actions in dqn_suggestions:
#     print(f"User {user_id}: {actions}")

print("\n🟢 A2C Model Suggestions:")
for user_id, actions in a2c_suggestions:
    print(f"User {user_id}: {actions}")


🟢 A2C Model Suggestions:
User 1: ['Watch educational videos']
User 2: ['Watch educational videos']
User 3: ['Watch educational videos']
User 4: ['Watch educational videos']
User 5: ['Watch educational videos']
User 6: ['Watch educational videos']
User 7: ['Watch educational videos']
User 8: ['Watch educational videos']
User 9: ['Watch educational videos']
User 10: ['Watch educational videos']
User 11: ['Watch educational videos']
User 12: ['Watch educational videos']
User 13: ['Watch educational videos']
User 14: ['Watch educational videos']
User 15: ['Watch educational videos']
User 16: ['Watch educational videos']
User 17: ['Watch educational videos']
User 18: ['Watch educational videos']
User 19: ['Watch educational videos']
User 20: ['Watch educational videos']
User 21: ['Watch educational videos']
User 22: ['Watch educational videos']
User 23: ['Watch educational videos']
User 24: ['Watch educational videos']
User 25: ['Watch educational videos']
User 26: ['Watch educational vide

# Modification

In [17]:
class MentalHealthEnv(gym.Env):
    def __init__(self, df):
        super(MentalHealthEnv, self).__init__()

        self.df = df.copy().reset_index(drop=True)
        self.current_index = 0

        # Define observation space: 7 scores
        self.observation_space = spaces.Box(low=0, high=300, shape=(7,), dtype=np.float32)

        # Define discrete action space
        self.actions = [
            "Engage in Outdoor activities",
            "Reduce the social media usage",
            "Watch educational videos",
            "Increase mindfulness practicess like yoga",
            "Interact with positive communities",
            "Avoid stress-inducing content",
            "Follow a structured daily routine"
        ]
        self.action_space = spaces.Discrete(len(self.actions))

        # Action effects
        self.action_effects = {
            "Engage in Outdoor activities": np.array([10, 5, 0, 8, -5, 7, 0]),
            "Reduce the social media usage": np.array([5, -15, 0, 0, 10, 0, 0]),
            "Watch educational videos": np.array([5, 0, 15, 7, -3, 10, 0]),
            "Increase mindfulness practicess like yoga": np.array([12, 0, 0, 5, -8, 15, 0]),
            "Interact with positive communities": np.array([8, 0, 0, 5, -5, 5, 0]),
            "Avoid stress-inducing content": np.array([5, 0, 0, -10, 10, 5, 0]),
            "Follow a structured daily routine": np.array([7, 0, 3, 6, -2, 8, 0])
        }

        self.state = self._get_state_from_row(self.current_index)

    def _get_state_from_row(self, index):
        return self.df.iloc[index, :7].values.astype(np.float32)

    def reset(self, seed=None, options=None):
        self.current_index = np.random.randint(0, len(self.df))
        self.state = self._get_state_from_row(self.current_index)
        return self.state, {}

    def step(self, action):
        action_str = self.actions[action]
        effect = self.action_effects[action_str]

        # Apply effect and clip
        self.state = np.clip(self.state + effect, 0, 300)

        # Reward can be based on improvements (e.g., sum of all scores)
        reward = np.sum(effect)

        # Simulate episode termination
        done = False
        if np.all(self.state >= 290):  # agent reached high scores
            done = True

        info = {}

        return self.state, reward, done, False, info
